In [ ]:
%matplotlib inline
import sys
sys.argv.append('-nomodels')
from summarizer_modules import str2vec_avg
import summarizer
import summarizer_modules
from summarizer_modules import *
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from importlib import reload



str2vec=[summarizer_modules.str2vec_avg, summarizer_modules.str2vec_arora, summarizer_modules.str2vec_doc2vec, summarizer_modules.str2vec_skipthought]
names = {str2vec_avg: 'SIF Average', str2vec_arora: 'Arora', str2vec_doc2vec: 'Paragraph Vectors', str2vec_skipthought: 'Skipthought'}

In [ ]:
import pickle
import matplotlib
from summarizer_modules import normalizeVec
from ducpreprocess import *
matplotlib.rcParams.update({'font.size': 18})
font1 = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
font2 = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 18}
corpus = pickle.load(open('data/corpus_new.pickle', 'rb'))
sentences = pickle.load(open('data/sent.pickle', 'rb'))
#Hacky fix
sentences = [(sent, [np.inner(normalizeVec(v1), normalizeVec(v2)) for v1, v2 in zip(emb, doc_emb)], words, [normalizeVec(v) for v in emb], [normalizeVec(v) for v in doc_emb], rouge) for sent, corr, words, emb, doc_emb, rouge in sentences]
ref = pickle.load(open('data/ref.pickle', 'rb'))
docs = pickle.load(open('data/docs.pickle', 'rb'))
#total_emb, total_sne = pickle.load(open('sne.pickle', 'rb'), encoding='latin1')

In [ ]:
wl = [x[2] for x in sentences]
corrs = [x[1] for x in sentences]

for i in range(len(str2vec)):
    function_name = names[str2vec[i]]
    print("Starting output for function: {}".format(function_name))
    corr = [x[i] for x in corrs]
    #Adjust to compensate
    wl_constant = .008163 
    
    #Flatter
    corr = [x[0] for x in corr]
    corr_adj = [corr[i] - wl_constant * wl[i] for i in range(len(corr))]
    plt.figure()
    plt.plot(np.log(wl), corr, 'bo', alpha=.05)
    p_initial = [1.0, 1, 0.1, 0.0]
    
    plt.plot(np.unique(np.log(wl)), np.poly1d(np.polyfit(np.log(wl), corr, 1))(np.unique(np.log(wl))))
    plt.xlabel("Log Word Length")
    plt.ylabel("Cosine Similarity with Document Vector")
    plt.title("Correlation v Word Length for {}".format(function_name))
    print("1D polynomial plot: {}".format(np.poly1d(np.polyfit(np.log(wl), corr, 1))))
    
    matplotlib.rcParams.update({'font.size': 18})
    plt.figure()
    n, bins, patches = plt.hist(corr, 50, normed=1, facecolor='green', alpha=0.75)
    from scipy.optimize import curve_fit
    from sklearn.metrics import r2_score
    #Code thanks to https://github.com/Ffisegydd/python-examples/blob/master/examples/scipy/fitting%20a%20gaussian%20with%20scipy%20curve_fit.py
    def gauss(x, *p):
        a, b, c = p
        y = a*np.exp(-np.power((x - b), 2.)/(2. * c**2.))
        return y
    p_initial = [1.0, 0.5, 0.1]
    popt, pcov = curve_fit(gauss, bins[:-1],n, p0=p_initial)
    y_fit = gauss(bins[:-1], *popt)
    plt.plot(bins[:-1], y_fit)
    plt.xlim([-.25, 1])
    plt.ylim([0, 10])
    plt.xlabel("Cosine Similarity", fontsize=18)
    plt.ylabel("Freq", fontsize=18)
    plt.tight_layout()
    plt.savefig('fig/dist_{}.png'.format(function_name))
    #plt.title("Correlation distribution for {}".format(function_name))
    
    #Output model parameters
    r2 = r2_score(n, y_fit)
    print("Gaussian model with mean {}, variance {} with r2={}".format(popt[1], popt[2], r2))
    
    plt.figure()
    
    n_adj, bins_adj, patches_adj = plt.hist(corr_adj, 50, normed=1, facecolor='green', alpha=0.75)
    
    #plt.xlabel("Correlation")
    #plt.ylabel("Freq")
    #plt.title("Correlation distribution for {}, after adjusting by {} per word".format(function_name, wl_constant))
    #_,_,_ = plt.hist([corr[i]for i in range(len(corr)) if wl[i] > 0 and wl[i] <=20], 50, normed=1, facecolor='green', alpha=0.75)
    #plt.figure()
    #_,_,_ = plt.hist([corr[i]for i in range(len(corr)) if wl[i] > 20 and wl[i] <=50], 50, normed=1, facecolor='green', alpha=0.75)
    #plt.figure()
    #_,_,_ = plt.hist([corr[i]for i in range(len(corr)) if wl[i] > 5 and wl[i] <=100], 50, normed=1, facecolor='green', alpha=0.75)

In [ ]:
from collections import Counter

wl = [x[2] for x in ref]
corrs_ref = [x[1] for x in ref]
corrs_sent = [x[1] for x in sentences]
corrs_sents = list((Counter(corrs_sent) - Counter(corrs_ref)).elements())
wl_sent = [x[2] for x in sentences]
for i in range(len(str2vec)):
    
    function_name = str2vec[i].__name__
    print("Starting output for function: {}".format(str2vec[i].__name__))
    corr_ref = [x[i] for x in corrs_ref]
    corr_sent = [x[i] for x in corrs_sent]
    print("Mean of {} compared to {}".format(np.mean(corr_ref), np.mean(corr_sent)))
    plt.figure()
    plt.plot(wl, corr_ref, 'g+')
    plt.plot(np.unique(wl), np.poly1d(np.polyfit(wl, corr_ref, 1))(np.unique(wl)))
    plt.title("Correlation distribution of optimal extractive summary for {}".format(function_name))
    
    plt.figure()
    n, bins, patches = plt.hist(corr_ref, 50, normed=1, facecolor='green', alpha=0.5)
    plt.xlim([0, 1])
    n, bins, patches = plt.hist(corr_sent, 50, normed=1, facecolor='blue', alpha=0.5)
    #plt.title("Histogram of correlation of all sentences for {}. Green = optimal extractive summary".format(function_name))
    plt.xlabel("Cosine Similarity")
    plt.ylabel("Freq")
    plt.figure()
    
    plt.plot(wl, corr_ref, 'g+')
    plt.plot(wl_sent, corr_sent, 'b+', alpha=(len(corr_ref) * 1.0/len(corr_sent)))
    plt.xlim([0, 30])
    
    
    corr_adj_ref = corr_ref - .09 * np.log(wl)
    corr_adj_sent = corr_sent - .09 * np.log(wl_sent)
    plt.figure()
    n, bins, patches = plt.hist(corr_adj_ref, 50, normed=1, facecolor='green', alpha=0.5)
    n, bins, patches = plt.hist(corr_adj_sent, 50, normed=1, facecolor='blue', alpha=0.5)
    plt.xlabel("Adjusted Cosine Similarity")
    plt.ylabel("Freq")
    #plt.title("Histogram of adjusted correlation of all sentences for {}. Green = optimal extractive summary".format(function_name))

In [ ]:
"""from sklearn.manifold import TSNE
import random

sent_emb = [x[3] for x in sentences]
ref_emb = [x[3] for x in ref]
doc_emb = [x[4] for x in sentences]

doc_emb_u = []
for vec in doc_emb:
    if not any(np.array_equal(vec, doc_emb_u) for vec_u in doc_emb_u):
        doc_emb_u.append(vec)
doc_emb = doc_emb_u

total_emb = sent_emb + ref_emb + doc_emb

model = TSNE(n_components=2, random_state=0)
total_sne = model.fit_transform(total_emb)

sent_sne = total_emb[0:len(sent_emb)]
ref_sne = total_emb[len(sent_emb):len(ref_emb)]
doc_sne = total_emb[len(ref_emb):len(doc_emb)]

#Remove duplicated from doc_sne
#As numpy arrays are hashable


sne_dict=dict()
for i in range(len(total_emb)):
    emb = total_emb[i]
    sne = total_sne[i]
    sne_dict[str(emb)]=sne
"""

In [ ]:
import random
import numpy as np
from sklearn.manifold import TSNE

def npindex(total_emb, vec):
    for i in range(len(total_emb)):
        v = total_emb[i]
        if np.allclose(v, vec):
            return i
    return None

for _ in range(10):
    
    v = random.choice(list(corpus.values()))
    docvec = summarizer_modules.str2vec_arora(' '.join(v[0]))
    
    sent_emb = [summarizer_modules.str2vec_doc2vec(x) - docvec for x in v[0]]
    ref_emb = [x for i, x in enumerate(sent_emb) if v[4][i]]
    sent_emb = [x for i, x in enumerate(sent_emb) if not v[4][i]]
    total_emb = sent_emb + ref_emb
    total_emb.append(docvec - docvec)
    
    model = TSNE(n_components=2, random_state=0)
    total_sne = model.fit_transform(total_emb)
    
    
    sent_x = total_sne[:len(sent_emb)][:, 0]
    sent_y = total_sne[:len(sent_emb)][:, 1]
    ref_x =  total_sne[len(sent_emb):-1][:, 0]
    ref_y =  total_sne[len(sent_emb):-1][:, 1]
    doc_x =  [total_sne[-1][0]]
    doc_y =  [total_sne[-1][1]]
    plt.figure()

    plt.plot(ref_x, ref_y, 'g+')
    
    plt.plot(sent_x, sent_y, 'b+')
    plt.plot(doc_x, doc_y, 'y+')
    plt.title("t-SNE visualization of randomly selected document. gold=doc, green=ref")

In [ ]:
sent_emb = np.asarray([x[3][1] for x in sentences])
ref_emb = np.asarray([x[3][1] for x in ref])

sent_emb = np.insert(sent_emb, 0, 0, axis=1)

ref_emb = np.insert(ref_emb, 0, 1, axis=1)
total_emb = np.concatenate([sent_emb, ref_emb])

np.savetxt('emb.txt', total_emb)

In [ ]:
sent_emb_diff = np.asarray([x[3][1]-x[4][1] for x in sentences])
ref_emb_diff = np.asarray([x[3][1]-x[4][1] for x in ref])

sent_emb_diff = np.insert(sent_emb, 0, 0, axis=1)

ref_emb_diff = np.insert(ref_emb, 0, 1, axis=1)
total_emb_diff = np.concatenate([sent_emb, ref_emb])

np.savetxt('emb_diff.txt', total_emb_diff)

In [ ]:
#Exploration of regression scores
def regression_scores(vec):
    return 2.15*vec[18] - 3.23 *vec[36] - 3.08*vec[90] -3.58*vec[159] + 2.54*vec[195] - 2.01*vec[249]

regg_scores = [regression_scores(vec) for vec in sent_emb_diff]
regg_scores_ref = [regression_scores(vec) for vec in ref_emb_diff]
print(len(sentences))

corr = [x[1] for x in corrs]
corr_ref = [x[1][1] for x in ref]

plt.figure()
plt.plot(corr, regg_scores, 'b+')
plt.plot(corr_ref, regg_scores_ref, 'g+')
plt.plot(np.unique(corr), np.poly1d(np.polyfit(corr, regg_scores, 1))(np.unique(corr)))
plt.xlim((0, 1))
plt.title("Regression score by correlation to document vector")
plt.xlabel("Correlation to document vector")
plt.ylabel("Regression score")

plt.figure()
_,_,_ = plt.hist(regg_scores, 50, normed=1, facecolor='blue', alpha=0.5)
_,_,_ = plt.hist(regg_scores_ref, 50, normed=1, facecolor='green', alpha=0.5)
plt.title("Histogram of regression score")
plt.xlabel("Regression score")
plt.ylabel("Freq")

In [ ]:
svm = pickle.load(open('svm.pickle', 'rb'), encoding='latin1')
import ducpreprocess
reload(ducpreprocess)
data_dev, labels_dev = ducpreprocess.get_sent_matrix(corpus, type=2)
predicts = [x[1] for x in svm.predict_proba(data_dev)]
predict_sent = [p for p, label in zip(predicts, labels_dev) if label == -1]
predict_ref = [p for p, label in zip(predicts, labels_dev) if label == 1]

plt.figure()
_,_,_ = plt.hist(predict_sent, 50, normed=1, facecolor='blue', alpha=0.5)
_,_,_ = plt.hist(predict_ref, 50, normed=1, facecolor='green', alpha=0.5)
plt.title("SVM predictions. Green = ref, blue = sent")
plt.xlabel("Predicted probability")
plt.ylabel("Freq")

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('vectors.bin', binary=True, limit=100000)
def find_word_for_dim(i):
    max=[]
    min=[]
    print("Dim: {}".format(i))
    for word in word_vectors.vocab.keys():
        vec=word_vectors[word]
        vec = vec / np.linalg.norm(vec)
        if vec[i] > .2:
            max.append(word)
        if vec[i] < -.2:
            min.append(word)   
        

find_word_for_dim(18)
find_word_for_dim(36)
find_word_for_dim(90)
find_word_for_dim(159)
find_word_for_dim(195)
find_word_for_dim(209)

In [ ]:
from summarizer_modules import str2vec_arora
from summarizer_modules import str2vec_doc2vec
corrs=[]
corrs_doc2vec=[]
for v in corpus.values():
    docvec = str2vec_arora((' '.join(v[0])))
    summvecs = [str2vec_arora((' '.join(summ))) for summ in v[1]]
    corrs.extend([np.inner(docvec, summvec) for summvec in summvecs])
    docvec = str2vec_doc2vec((' '.join(v[0])))
    summvecs = [str2vec_doc2vec((' '.join(summ))) for summ in v[1]]
    
    corrs_doc2vec.extend([np.inner(docvec, summvec) for summvec in summvecs])
plt.figure()
plt.hist(corrs)
#plt.title("Correlation of reference summary to document vector")
plt.xlabel("Cosine Score (Arora)")
plt.ylabel("Freq")

plt.figure()
plt.hist(corrs_doc2vec)
#plt.title("Correlation of reference summary to document vector")
plt.xlabel("Cosine Score (Paragraph Vectors)")
plt.ylabel("Freq")

In [ ]:
from sklearn.svm import SVC
from sklearn import linear_model

m = np.loadtxt('rougematrix.txt')
optimal = [0.31, 0.56863, 0.34906, 0.57, 0.57692, 0.75248, 0.63366, 0.31868, 0.48077, 0.59804, 0.67327, 0.625, 0.52475, 0.39216, 0.34, 0.53, 0.56436, 0.42157, 0.62637, 0.46, 0.58654, 0.61321, 0.40594, 0.62745, 0.85915, 0.56604, 0.32653, 0.53125, 0.55, 0.59406, 0.57426, 0.53535, 0.54545, 0.48039, 0.55882, 0.38384, 0.57282, 0.45098, 0.52083, 0.49474, 0.46, 0.36408, 0.64706, 0.67677, 0.55882, 0.32, 0.65385, 0.65347, 0.48515, 0.72549, 0.53, 0.57, 0.26263, 0.60577, 0.5625, 0.36893, 0.52, 0.44554, 0.48077, 0.59406, 0.51515, 0.34951, 0.70297, 0.64, 0.54369, 0.68687, 0.52525, 0.52525, 0.57843, 0.58, 0.64078, 0.25243, 0.62136, 0.76923, 0.32, 0.3871, 0.58, 0.54455, 0.31633, 0.67677, 0.48485, 0.46535, 0.37143, 0.67961, 0.36634, 0.55, 0.40594, 0.52381, 0.44231, 0.75, 0.52941, 0.55882, 0.54808, 0.63551, 0.75238, 0.43, 0.68657, 0.44444, 0.63366, 0.52941, 0.50495, 0.58824, 0.5, 0.36538, 0.43137, 0.69307, 0.51961, 0.42, 0.58095, 0.68421, 0.47475, 0.41, 0.57944, 0.41346, 0.41667, 0.58252, 0.46465, 0.54, 0.41414, 0.45545, 0.47573, 0.5534, 0.66, 0.39216, 0.52885, 0.56311, 0.50962, 0.63725, 0.53, 0.44118, 0.62376, 0.48148, 0.51429, 0.54, 0.3, 0.61321, 0.68317, 0.5, 0.64646, 0.39, 0.74747, 0.50495, 0.43, 0.38835, 0.54717, 0.48, 0.31633, 0.48485, 0.54902, 0.73737, 0.56, 0.61905, 0.59406, 0.46, 0.71569, 0.72464, 0.64646, 0.68868, 0.56311, 0.52, 0.54902, 0.52475, 0.5102, 0.64706, 0.41346, 0.34375, 0.67606, 0.63462, 0.54286, 0.53535, 0.45455, 0.41176, 0.40196, 0.5, 0.59406, 0.66667, 0.59406, 0.6699, 0.61165, 0.61, 0.47, 0.67647, 0.4902, 0.40196, 0.66667, 0.40404, 0.60215, 0.70297, 0.30769, 0.54082, 0.37624, 0.69307, 0.38384, 0.7973, 0.58163, 0.37624, 0.55, 0.41237, 0.48039, 0.38614, 0.6, 0.63107, 0.34314, 0.54902, 0.48544, 0.73737, 0.59596, 0.59223, 0.53, 0.5534, 0.26733, 0.90196, 0.63, 0.54717, 0.37, 0.51429, 0.55882, 0.59794, 0.59, 0.75248, 0.4, 0.50495, 0.66667, 0.59184, 0.61765, 0.52, 0.55, 0.66337, 0.50495, 0.55102, 0.59, 0.41584, 0.77228, 0.60976, 0.48039, 0.56566, 0.54545, 0.45545, 0.59406, 0.45455, 0.56311, 0.59406, 0.39394, 0.51515, 0.65347, 0.7451, 0.625, 0.27451, 0.42424, 0.55882, 0.35165, 0.51429]
print(np.corrcoef(m))
maxes = ([max(x) for x in m.T])
print(sum(maxes)/len(maxes))
labels = [max(range(len(x)), key=lambda i: x[i]) for x in m.T]
avg_cos = np.asarray([np.mean(sum([x[1] for x in doc[3]],[])) for doc in docs])
features = avg_cos.reshape(-1, 1)
svm = SVC()
svm.fit(features, labels)
print("SVM Accuracy: {}".format(svm.score(features, labels)))
print("Avg Cosine Correlation")
for x in m:
    print(np.corrcoef(x, avg_cos)[1][0])
print(np.corrcoef(optimal, avg_cos)[1][0])
print([np.mean([cos for label, cos in zip(labels, avg_cos) if label == i]) for i in range(6)])
print(labels.count(0))

masks = []

for x in m:
    plt.figure()
    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(avg_cos.reshape(-1, 1), x)
    plt.ylim([0, .6])
    plt.xlim([0, 1])
    inlier_mask = model_ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    
    line_X = np.arange(-5, 5)
    line_y_ransac = model_ransac.predict(line_X[:, np.newaxis])
    
    plt.scatter(avg_cos[inlier_mask], x[inlier_mask], color='green')
    plt.scatter(avg_cos[outlier_mask], x[outlier_mask], color='red')
    plt.plot(line_X, line_y_ransac, color='navy')
    print(model_ransac.estimator_.coef_)
    masks.append([1 if b else 0 for b in inlier_mask])
masks = np.asmatrix(masks)
print(np.corrcoef(masks))
print(masks)
plt.figure()
plt.scatter(avg_cos, optimal)
for x in masks[0,]:
    print(x)
print(len([x for x in masks[0,] if x==1]))
print(len([x for x in masks[3,] if x==1]))

In [ ]:
import random

for _ in range(10):
    doc = random.choice(docs)
    sent = random.choice(doc[3])
    sent_i = doc[3].index(sent)
    embs = list(doc[4])
    sent_emb = embs[sent_i]
    del embs[sent_i]
    plt_x = [np.inner(doc[2][method], emb[method]) for emb in embs]
    plt_y = [np.inner(sent_emb[method], emb[method]) for emb in embs]
    plt.figure()
    plt.scatter(plt_x, plt_y)
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.plot([0, 1], [0, 1])
    plt.xlabel("Doc Correlation")
    plt.ylabel("Sent Correlation")

In [ ]:
method = 1

ref_sents = [x[0] for x in ref]
#print(ref_sents)
for method in range(4):

    plt_x=[]
    plt_y=[]
    
    plt_x_ref=[]
    plt_y_ref=[]
    for doc in docs:
        embs = list(doc[4])
        sent_i = max(range(len(embs)), key=lambda i: np.inner(doc[2][method], embs[i][method]))
        sent_emb = embs[sent_i]
        sent = doc[3][sent_i]
        del embs[sent_i]
        x, y = None, None
        if sent[0] in ref_sents:
            x, y = plt_x_ref, plt_y_ref
        else:
            x, y = plt_x, plt_y
        x.extend([np.inner(doc[2][method], emb[method]) for emb in embs])
        y.extend([np.inner(sent_emb[method], emb[method]) for emb in embs])
    plt.figure()
    plt.scatter(plt_x_ref, plt_y_ref, alpha=.1, color='g')
    plt.scatter(plt_x, plt_y, alpha=.01)
    
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    
    for i in range(-10, 10):
        slope = 3
        plt.plot([0, 1], [ -i * .1 - 1, -i* .1 - 1 + slope], color="grey", alpha = .05 * (i + 10))
    plt.xlabel("Doc Correlation")
    plt.ylabel("Sent Correlation")
    poly = np.poly1d(np.polyfit(plt_x, plt_y, 2 if method < 2 else 1))
    plt.plot(np.unique(plt_x), poly(np.unique(plt_x)))
    print(poly)
    resid = [poly(x) - y for x, y in zip(plt_x, plt_y)]
    resid_ref = [poly(x) - y for x, y in zip(plt_x_ref, plt_y_ref)]
    print(np.std(resid))
    print(np.std(resid_ref))
    plt.figure()
    plt.scatter(plt_x, resid, alpha=.01)
    
    plt.plot([0, 1], [0, 0])
    #plt.scatter(plt_x_ref, resid_ref, alpha=.1, color='g')
    plt.figure()
    plt.hist(resid, alpha=.5, normed=1)
    plt.hist(resid_ref, alpha=.5, color='g', normed=1)
    plt.figure()
    plt.hist(plt_y, normed=1, alpha=.5)
    plt.hist(plt_y_ref, color='g', normed=1, alpha=.5)
    plt.xlim([-1, 1])
    plt.figure()

In [ ]:
#Performance of Near_redundancy under different factors
x = [0,    .5,   1    ,2,    3  ,  5,    10, 20, 30]
y = [32.8, 35.7, 35.9 ,36.3,36.6,  36.7, 36.6, 36.9, 36.7]
plt.figure
plt.plot(x, y)

In [ ]:
tfidf_dist = [.00, 0.45, 0.02, 0.17, 0.03, 0.22, 0.03, 0.28, 0.06, 0.06, 0.00, 0.45, 0.16, 0.27, 0.03, 0.19, 0.03, 0.21, 0.03, 0.15, 0.00, 0.02, 0.16, 0.03, 0.00, 0.01, 0.03, 0.04, 0.00, 0.01, 0.00, 0.17, 0.27, 0.03, 0.01, 0.16, 0.28, 0.17, 0.00, 0.09, 0.01, 0.03, 0.03, 0.00, 0.01, 0.29, 0.05, 0.15, 0.20, 0.04, 0.18, 0.22, 0.19, 0.01, 0.16, 0.29, 0.05, 0.29, 0.04, 0.20, 0.03, 0.03, 0.03, 0.03, 0.28, 0.05, 0.05, 0.06, 0.00, 0.00, 0.01, 0.28, 0.21, 0.04, 0.17, 0.15, 0.29, 0.06, 0.25, 0.20, 0.17, 0.06, 0.03, 0.00, 0.00, 0.20, 0.04, 0.00, 0.25,  0.26, 0.38, 0.06, 0.15, 0.01, 0.09, 0.04, 0.20, 0.00, 0.20, 0.26, 0.12, 0.00, 0.00, 0.00, 0.01, 0.18, 0.03, 0.01, 0.17, 0.38, 0.12]
plt.figure()
plt.hist(tfidf_dist)

plt.xlim([0, .6])
emb_type=0
import random
import itertools

cluster = random.choice(docs)
emb_dist = []
for s1, s2 in itertools.combinations(cluster[4], 2):
    emb_dist.append(np.inner(s1[emb_type], s2[emb_type]))
plt.figure()
plt.hist(emb_dist)
plt.xlim([0, .6])

from gensim.models import tfidfmodel

In [ ]:
greedy_cos_scores = [0.74805345893954633, 0.70637976872275343, 0.62496437445813569, 0.4334209944080411, 0.70124634438579425, 0.61991639425039702, 0.58800298395081374, 0.69736628839131753, 0.60365352281222595, 0.57741229306811503, 0.64743786523159008, 0.76335362959564435, 0.57691120632658421, 0.72941577553457104, 0.61820927575472251, 0.86910219741690342, 0.79768167412885338, 0.47022251542645971, 0.41597806515626784, 0.81140264648559313, 0.71241097495719663, 0.75825712074562068, 0.64289654563394127, 0.79083484132876447, 0.70767839654006781, 0.49567777096426807, 0.69909407047349703, 0.68071252305126673, 0.47744900403403834, 0.42542610043052775, 0.4789790492977849, 0.84244861506099711, 0.76118695031424333, 0.6902159315017552, 0.78612116346731054, 0.76891461920094162, 0.60395920754549903, 0.67169277571440622, 0.75919771507186107, 0.70635938091267547, 0.69783423521716681, 0.52343548248333105, 0.72770518974699472, 0.69412431614662518, 0.51223589127902858, 0.35363154610912306, 0.77429485339772608, 0.68504429449462889, 0.54612961407776595, 0.87833877263407256, 0.78093127353164515, 0.82225197106416337, 0.69962236660273014, 0.67033619513213794, 0.67649759016163269, 0.60253453427235348, 0.82988457552034534, 0.61375816444206199, 0.70616838554018402, 0.52864307094596297, 0.73226713931481757, 0.58602862229044239, 0.53420207968607603, 0.48772089022649723, 0.7864774544756129, 0.77417381403190266, 0.70146184090238395, 0.7967922634244109, 0.79087826239080616, 0.7356620582582527, 0.78240941344056225, 0.79262743486614684, 0.72108927118758914, 0.68537404815947212, 0.78110063753417514, 0.73122510263561913, 0.65448932835287321, 0.7593996164548259, 0.67162227338141089, 0.58651995321016959, 0.3224867191957933, 0.81462285647927779, 0.74169570040155197, 0.70763119569826027, 0.68210411085746625, 0.55945347337764384, 0.67918745157689242, 0.42375389051969653, 0.568246352095231, 0.42929402446222403]
plt.figure()
plt.hist(greedy_cos_scores)
plt.xlabel("Cosine Similarity", fontsize=18)
plt.ylabel("Freq", fontsize=18)
plt.tight_layout()
plt.savefig('fig/greedysentcos.png')

In [ ]:
first_component = [-0.09762673,-0.01984626,-0.05386575,0.0875482,0.03618532,-0.04648532,-0.01533625,-0.04439673,0.07073691,0.02020859,-0.12665796,-0.03097556,-0.07227197,-0.02738396,0.05481791,-0.00506371,-0.01626564,0.03369868,0.03080177,0.01819133,0.06875614,0.0355786,0.04019368,-0.02818428,-0.02209094,0.05418129,0.0496157,0.0614819,-0.02591645,0.0472538,0.04128136,-0.00290736,-0.0039223,-0.0234733,-0.02163736,0.05252197,-0.09156871,0.03279637,0.04419556,-0.03698228,-0.05733967,-0.07474401,0.1606355,0.13070298,0.05926913,0.00890097,0.01876796,-0.07465753,-0.14765179,-0.00644881,0.0543977,-0.00874069,0.05205558,0.08140484,-0.01471067,-0.03591435,-0.1117754,0.0027264,0.09674388,-0.10013767,-0.12041047,-0.06913955,-0.03560099,-0.00270038,0.00995046,-0.06464013,-0.05213869,0.02336153,0.0393586,0.06920559,0.02922447,0.02527685,0.02663035,0.04412861,0.023189,-0.00531015,0.03499466,-0.02423968,0.01136961,-0.01380582,-0.03807703,0.02360628,0.03851788,-0.06932288,0.0769464,0.00535265,-0.04384463,-0.05005166,-0.04330126,0.09560347,0.02492301,-0.09894897,0.02273585,-0.01370988,-0.08040128,0.03391581,-0.06349891,-0.06862244,0.07601657,0.00676117,0.00139526,0.08337108,0.00051518,-0.00025291,-0.08282007,0.03407196,-0.17093423,-0.00511782,0.01074712,-0.10302095,-0.00737771,-0.03992952,-0.00044895,-0.06951664,0.11931398,-0.02049736,-0.06313224,-0.03844705,-0.00489895,-0.03092416,0.02238491,-0.10429512,-0.06685176,-0.01177102,0.04167317,0.04107572,-0.06963748,0.02915489,0.03513566,-0.0593385,-0.0212302,0.03398347,0.05275497,-0.02136662,-0.10148652,-0.0344013,-0.04459528,-0.00774491,0.04471264,0.01958932,0.06461796,-0.01547985,0.04960655,-0.05903677,-0.08955337,-0.01586336,0.09545738,-0.02332863,0.05192729,-0.02276518,0.01883432,-0.04716429,-0.00767016,-0.03827581,-0.00062703,-0.15791895,0.06149467,0.03548422,-0.05813857,-0.05448442,0.0453456,0.00344903,-0.1046012,-0.00344763,0.12753533,0.01332342,0.04642873,0.04913153,-0.02122613,0.0741794,-0.06545443,-0.04684743,-0.00265002,0.0566631,0.01683476,-0.01546004,0.08088695,-0.15646161,0.14209595,-0.02181382,-0.02300327,-0.07433104,-0.00876267,0.00260114,-0.0192266,0.10906026,0.04507465,-0.00626048,0.07338181,-0.05736074,0.05513703,0.04503881,-0.06816687,0.0243422,0.03547794,-0.01726,0.01718868,0.04805279,0.05797942,-0.00779116,-0.10181751,0.06559385,0.02492503,-0.08911273,-0.02075531,-0.01002261,0.04289727,0.03205456,-0.07117203,0.04831832,-0.01296153,0.02883282,-0.10502579,-0.05724643,-0.04372814,-0.04136519,0.01853343,-0.11929604,-0.04303174,0.0686698,-0.0066496,-0.01259159,0.02976217,0.06093101,-0.01785665,-0.04624248,0.0552846,0.00944272,0.0167846,0.08543814,0.03427679,-0.01079003,-0.00147623,-0.08670206,-0.00690788,0.08451349,0.01586759,0.0612824,0.02713893,0.07639873,0.06083708,0.05802068,-0.00254352,0.0508127,-0.04894757,0.03806672,0.01008171,0.0284182,-0.04429965,-0.04302035,0.06045836,0.08170388,0.07634532,0.04896196,0.03341781,-0.08260471,-0.02983155,0.10147369,0.07092215,0.0102002,0.07203048,-0.01454441,0.1640545,-0.00558674,-0.00337068,-0.00916714,0.04089899,0.07498604,-0.00482272,0.02914961,-0.06887,-0.05074636,0.02478703,-0.00041836,0.00455064,-0.0132185,0.02347787,0.02456291,-0.07724374,0.16579521,-0.07664108,-0.0318883,0.02944632,0.06391358,-0.02223495,-0.00183343,0.00056177,-0.02670676,-0.02675089,0.03784881,0.0335072,0.04893145,-0.0205906,0.09266761,0.0147099,-0.05617354,-0.13493886,0.00267046,-0.02636754,-0.01502137]
plt.figure()
plt.hist(first_component)

In [ ]:
cos_diffs = [-0.067882125,0.053380934,0.010769519,0.002085952,-0.096241287,0.03914522,0.076382442,-0.037691666,-0.020859525,-0.026677522,-0.002179982,-0.036296467,-0.061550634,0.023925508,0.003239357,-0.00754356,-0.034443311,-0.004366877,0.042336396,0.031532854,0.023688931,0.01413672,-0.014818574,-0.058639562]
rouge_diffs = [-0.07005,0.00713,-0.14742,-0.15196,0.00249,-0.01457,0.01975,-0.01238,0.01707,-0.07575,-0.11279,0.03695,-0.00241,-0.01703,-0.00969,-0.12439,-0.06666,-0.01737,0.02963,-0.14461,0.03125,-0.00246,0.06308,0.02722]
stds = [0.00693573353054,0.0212620338786,0.0147888275491,0.00766485676855,0.017819303076,0.0155328625242,0.0213506362369,0.0105406831475,0.0220409035137,0.0106407149759,0.0100319932595,0.0122151659129,0.0189288699002,0.0132770997897,0.0148236149809,0.00597785877005,0.0135271160006,0.030711173121,0.0135775204053,0.0111175230566,0.0140648399388,0.0116004250319,0.0234822505865,0.00941222336991]
plt.figure()
plt.scatter(stds, rouge_diffs)
plt.plot([0, .04], [0, 0])
print(np.corrcoef(stds, rouge_diffs)[0, 1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def rand_jitter(arr):
    stdev = .02*(max(arr)-min(arr))
    return arr + np.random.randn(len(arr)) * stdev

def jitter(x, y, s=20, c='b', marker='o', cmap=None, norm=None, vmin=None, vmax=None, alpha=None, linewidths=None, verts=None, hold=None, **kwargs):
    return plt.scatter(rand_jitter(x), rand_jitter(y), s=s, c=c, marker=marker, cmap=cmap, norm=norm, vmin=vmin, vmax=vmax, alpha=alpha, linewidths=linewidths, verts=verts, hold=hold, **kwargs)

rouge_by_sent = [0.0942, 0.19082, 0.17633, 0.11594, 0.08213, 0.16667, 0.13043, 0.06763, 0.14734, 0.13043, 0.1401, 0.04589, 0.07488, 0.07729, 0.10386, 0.07488, 0.12319, 0.14496, 0.06388, 0.18919, 0.19902, 0.06143, 0.11302, 0.15971, 0.12776, 0.11548, 0.08354, 0.13759, 0.18919, 0.04177, 0.10074, 0.12019, 0.12981, 0.1274, 0.16587, 0.14904, 0.11538, 0.16346, 0.08654, 0.10096, 0.21154, 0.05288, 0.15625, 0.1274, 0.09135, 0.0625, 0.09375, 0.1274, 0.11779, 0.26536, 0.27764, 0.32678, 0.16953, 0.13268, 0.10811, 0.20393, 0.1769, 0.11057, 0.086, 0.14742, 0.13514, 0.11548, 0.16465, 0.1477, 0.03874, 0.13801, 0.15496, 0.10896, 0.11864, 0.13075, 0.15254, 0.15981, 0.15981, 0.15981, 0.14286, 0.1138, 0.14951, 0.1152, 0.10784, 0.15931, 0.05147, 0.12745, 0.14951, 0.09559, 0.10294, 0.06127, 0.09314, 0.13235, 0.1299, 0.04167, 0.11765, 0.1152, 0.19221, 0.17518, 0.16788, 0.14355, 0.13139, 0.17032, 0.20681, 0.16058, 0.09002, 0.18248, 0.09489, 0.16058, 0.09976, 0.09976, 0.21814, 0.2598, 0.14706, 0.11029, 0.11029, 0.09804, 0.14461, 0.15441, 0.07598, 0.10784, 0.22304, 0.15479, 0.13022, 0.13022, 0.20393, 0.15971, 0.10074, 0.11302, 0.17936, 0.07617, 0.12776, 0.08108, 0.13268, 0.11548, 0.08354, 0.12039, 0.17327, 0.11386, 0.10644, 0.17327, 0.19307, 0.11634, 0.08168, 0.16337, 0.11386, 0.17079, 0.14356, 0.18812, 0.16089, 0.12343, 0.13602, 0.10831, 0.1738, 0.10831, 0.09824, 0.08312, 0.12091, 0.10327, 0.05542, 0.15365, 0.17884, 0.14358, 0.0881, 0.14524, 0.08333, 0.12857, 0.13095, 0.09524, 0.14762, 0.1119, 0.09524, 0.09524, 0.11667, 0.10476, 0.1, 0.12381, 0.05238, 0.07857, 0.08333, 0.10952, 0.13012, 0.16867, 0.10602, 0.15663, 0.08434, 0.11084, 0.13253, 0.12771, 0.16867, 0.08193, 0.18795, 0.17349, 0.13012, 0.13253, 0.11566, 0.08916, 0.1561, 0.06829, 0.16829, 0.07805, 0.14878, 0.16341, 0.07805, 0.06585, 0.05366, 0.15366, 0.03415, 0.04146, 0.11951, 0.15366, 0.07073, 0.09512, 0.1561, 0.11307, 0.07286, 0.09799, 0.12563, 0.07538, 0.1005, 0.11055, 0.08543, 0.0804, 0.10302, 0.11307, 0.0804, 0.16332, 0.07538, 0.12563, 0.07538, 0.13568, 0.19608, 0.20098, 0.07353, 0.16667, 0.15441, 0.15931, 0.13971, 0.1152, 0.14461, 0.1201, 0.08333, 0.05147, 0.11275, 0.10294, 0.18873, 0.24755, 0.17892, 0.09314, 0.10784, 0.23039, 0.18873, 0.03676, 0.14461, 0.06618, 0.14461, 0.19363, 0.07353, 0.1348, 0.14951, 0.11029, 0.14112, 0.15328, 0.15328, 0.16302, 0.25791, 0.0146, 0.11922, 0.12895, 0.07299, 0.16058, 0.13625, 0.16058, 0.17762, 0.09732, 0.08029, 0.07299, 0.21411, 0.07299, 0.15815, 0.15789, 0.13033, 0.17293, 0.11028, 0.12531, 0.16291, 0.15539, 0.12531, 0.06767, 0.1203, 0.08521, 0.13784, 0.15288, 0.13534, 0.08271, 0.11779, 0.07268, 0.04511, 0.13283, 0.15981, 0.10654, 0.09927, 0.18644, 0.13559, 0.15738, 0.06538, 0.09443, 0.10169, 0.07022, 0.12349, 0.10169, 0.08717, 0.08959, 0.1816, 0.06538, 0.07022, 0.12352, 0.12352, 0.10451, 0.20665, 0.17815, 0.09026, 0.11164, 0.20665, 0.14252, 0.10689, 0.13302, 0.08076, 0.08789, 0.19471, 0.15865, 0.12019, 0.23077, 0.14904, 0.16827, 0.17548, 0.07692, 0.17548, 0.13702, 0.17788, 0.07933, 0.08911, 0.10891, 0.2104, 0.09653, 0.17327, 0.09901, 0.08416, 0.29703, 0.13119, 0.13366, 0.15347, 0.0495, 0.05198, 0.16584, 0.07426, 0.13614, 0.17822, 0.14109, 0.15347, 0.09901, 0.11386, 0.15099, 0.11386, 0.18317, 0.11634, 0.18812, 0.04703, 0.12129, 0.13896, 0.16377, 0.13648, 0.10174, 0.15136, 0.134, 0.13648, 0.07692, 0.13648, 0.09181, 0.08933, 0.04218, 0.05211, 0.09181, 0.08685, 0.08189, 0.1464]
marginal_rouge_by_sent = [0.0942, 0.17149999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14496, 0.05160000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12019, 0.11297999999999998, 0.026439999999999964, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26536, 0.16953000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16465, 0.11138, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14951, 0.09559000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19221, 0.15086, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21814, 0.14215, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15479, 0.11793999999999996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17327, 0.09901000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12343, 0.08816, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0881, 0.11428, 0.05238000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13012, 0.14939999999999998, 0.0048200000000000465, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1561, 0.05853, 0.04634000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11307, 0.05778, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19608, 0.16667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18873, 0.22058, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14112, 0.13869, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15789, 0.12281, 0.01754, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15981, 0.09200999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12352, 0.10451, 0.03563000000000005, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19471, 0.12980999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08911, 0.09901000000000001, 0.06930000000000003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16584, 0.05446000000000001, 0.06930999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13896, 0.1464, 0.007450000000000012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cos_by_sent =[0.0024752868756593295, 0.0023187767779675032, 0.0022353540983807594, 0.002103243309073145, 0.00209225907575922, 0.0021155454537804866, 0.0019057512288584188, 0.0010989072663399732, 0.0019810890287735041, 0.0020122450523029807, 0.0013585465241293673, 0.0014646410173915039, 0.0015016662121553532, 0.0023382847763178602, 0.0016924100655296337, 0.0013590680994327404, 0.0010394697979788973, 0.001542717776587962, 0.00094493278357856152, 0.0013209541566251376, 0.0013448817606378704, 0.00096032181765488952, 0.0015141398995312746, 0.0009636120328530409, 0.0010310483719391435, 0.00095996366184548465, 0.0010205918886446242, 0.0012685826115867644, 0.0007393521641574026, 0.00089893347421357236, 0.00071485000598417773, 0.0045974716983766886, 0.0043361833487613891, 0.0034304069336632318, 0.004335341376670996, 0.0033383058985168167, 0.0036836071980671931, 0.0042250034696870069, 0.0027649744897108924, 0.0031015896252822132, 0.0034570666355019926, 0.0024343155547431809, 0.0028005526697061772, 0.0023551926491089483, 0.0039333704112879449, 0.0024694627552183713, 0.0017751341016461228, 0.0032191766063806326, 0.0035392218708799146, 0.0049962882949462304, 0.0046001159412018501, 0.0045476408047088806, 0.0038595810055628713, 0.0040554999963547753, 0.0040752459203897565, 0.0041495764086661993, 0.0042114777055316738, 0.0022242802001417048, 0.0036897568077992209, 0.0046404400029552914, 0.0035228615873452778, 0.0044603354269027616, 0.001457324470139834, 0.0013483752222436616, 0.0012720969397170213, 0.00092825806407149805, 0.0010977657018973643, 0.0013768037031765066, 0.001355152967153176, 0.000190361774925748, 0.0012975769008113102, 0.0010184447861670491, 0.00090646352866106722, 0.0011519509848954175, 0.0010354472999165331, 0.00087718004659917339, 0.0028788430888105018, 0.0026807594821247013, 0.0027985427376251478, 0.0024898970281691415, 0.0026123545738274659, 0.0019768263817545148, 0.0022341212826105213, 0.00047323516779030012, 0.0021254136980560358, 0.0025777907101815719, 0.0021699462851864082, 0.0012726675172319594, 0.0027850140581414389, 0.00096887789104938872, 0.00247511356729489, 0.0010798938362916453, 0.0024718284186303462, 0.0020542052973569359, 0.0020561545219308897, 0.0019406495397618264, 0.0021881752707736402, 0.0017635770689967208, 0.0021429358256642829, 0.0018599720048592143, 0.0020345887064855566, 0.0011480701652914721, 0.0019193112319822979, 0.0018439021271515, 0.0018360772410379326, 0.0017326980024027746, 0.0031712564304743371, 0.00306082903183641, 0.002904372675023617, 0.0015564676967168704, 0.0022538607997479639, 0.0018847443699538725, 0.0020051604520920779, 0.0021529428377394353, 0.0023928632745674489, 0.0016305283089877859, 0.0028248760271969561, 0.0038670516504686076, 0.0035834366644406741, 0.003535852944103354, 0.0030301490290141732, 0.0026422046573784633, 0.0032978397755006471, 0.0034571121987300802, 0.0034832479303008555, 0.0022884338041317676, 0.0028441935612098677, 0.0018137466765602993, 0.0025154294527030686, 0.0029968113925226635, 0.0017348646533982229, 0.0025260057668479866, 0.0015995026102957745, 0.0013666312542682574, 0.001480236067339381, 0.0012831733586831759, 0.0013608015675551505, 0.001399589425437343, 0.00063967388668818577, 0.0015547055423066869, 0.0013438784959461569, 0.0012311841247301311, 0.00064025472841218781, 0.0015150298222510361, 0.0010469945246646046, 0.0017471150942828439, 0.0016892141488533397, 0.0017172456320559116, 0.0014131867180441811, 0.0013648589994816409, 0.0014233042228563194, 0.0015289563340880848, 0.0014023774883840218, 0.0015796619763349797, 0.0015645003402750093, 0.0017130799662039621, 0.0014312130287542104, 0.0013621443718719694, 0.0040310986182429138, 0.0039959025737102069, 0.0029802384193820331, 0.0027322096667762941, 0.0039279433327577551, 0.0031126787080108446, 0.0030145039906626292, 0.0029966492203980771, 0.0018779113504127304, 0.0023463304388715227, 0.0028494490700101544, 0.0021586860873133657, 0.0032887936482750026, 0.0023218822395526476, 0.0023885355046303763, 0.0024355048414802013, 0.0026124135517570975, 0.002993744321765651, 0.0030608657220768348, 0.0026237799611386835, 0.0028191662283611188, 0.0024699566995730854, 0.0021494698754815455, 0.0026994354707146253, 0.0029468700956892871, 0.0027868939244372749, 0.0021540795951765059, 0.00222911600075242, 0.001680864864219768, 0.0026164031812222844, 0.0028025826671681407, 0.0021439422357957417, 0.0023998750702703547, 0.0024381775857592893, 0.0040810502101600934, 0.0037374195665867724, 0.0037569396092258299, 0.0031624158261344998, 0.0040615921767384253, 0.0036566714280412129, 0.0032857809670842853, 0.0012814430121236089, 0.0027584600939535983, 0.0018028897383759843, 0.0025019178107851411, 0.0012583920994271305, 0.0034939801227593745, 0.0020854822266022782, 0.0025936067113342654, 0.0022790750037712157, 0.003787249293604487, 0.0050483417144734531, 0.0043872229709132054, 0.0044106043872813592, 0.0027746934578361153, 0.0037498899934431873, 0.0037803510841591826, 0.0015881242150905122, 0.0028285561636218772, 0.0034289392231446609, 0.0024584351424619426, 0.0050483417144734531, 0.0029306888835574124, 0.0045412053187029134, 0.0018016816211356697, 0.00355597641565458, 0.001554955123798372, 0.0021771670438231103, 0.0020068008049814192, 0.00182463483309381, 0.0017941265221533846, 0.0012121885511908791, 0.001474409583930441, 0.00069306699975471163, 0.0014398073683547568, 0.0013419889937835623, 0.0019252283997392265, 0.0011243440043742635, 0.0010606627948441145, 0.0017243035991119176, 0.00053957877828807112, 0.0013546950155433606, 0.0036616843055760294, 0.003265033870745079, 0.0033231637074666262, 0.0020232980587966795, 0.0032808264131717036, 0.0028284721086042101, 0.0029609635880289147, 0.0025543546663333158, 0.0036216837691295891, 0.0017064715972681227, 0.002251637425282043, 0.0022425963690220095, 0.0031640089587486719, 0.0019268423264645759, 0.0020743759454225795, 0.0012463389688792267, 0.0030898135012092003, 0.0018115849401568079, 0.0029410070324465032, 0.0025653587072620894, 0.0030790627576385201, 0.0016180370988322815, 0.0011904109071959209, 0.0026288734482832823, 0.0021934706924614262, 0.0018338960135740362, 0.0016326360709422621, 0.0027102791734125871, 0.0013425738005031717, 0.0025956156258972701, 0.0024118826918563528, 0.0015840922188039585, 0.0029465742501795914, 0.0014684294567229987, 0.0015531606569695565, 0.0048017489430879686, 0.0032655460456625356, 0.0047961159906916382, 0.0030398087847803299, 0.0042387410845488903, 0.0033891424201502691, 0.0033911667390032823, 0.0025613978420440248, 0.0042743491477541555, 0.0030750842245245973, 0.0021445336036757861, 0.0028669645568577197, 0.0036015701927820253, 0.0043624028239588287, 0.0023208492766128891, 0.0023846755890326891, 0.0034754324219324869, 0.0024135758323761722, 0.0040107477275857142, 0.0016061761358209864, 0.0014581905990533237, 0.001171022883055282, 0.0012919345746485356, 0.0010033668760291124, 0.001369753718073613, 0.0015076935649069641, 0.0012441845365657169, 0.0011348742202638823, 0.0012173052963830339, 0.0011619742278200825, 0.0015518585214806349, 0.0010983858360703992, 0.00047145352905195859, 0.0015198611696031338, 0.00072030130573995062, 0.00089727865286531633, 0.0026828531994983877, 0.0026489211978048262, 0.0019970940458511601, 0.0026351753995834939, 0.0023157673586339232, 0.0024085826626844007, 0.0019972292827712926, 0.0021210278844953742, 0.001937823575049508, 0.0020754122192119554, 0.0017885327715697001, 0.0026763069708405267, 0.0019694819321518483, 0.002480084636494503, 0.0024317084920058209, 0.0023319895025829276, 0.0022189930672119103, 0.0023170544993504493, 0.0015560301369122028, 0.001190307910726138, 0.0021101749884969768, 0.0021196528147372782, 0.0014991965643185515, 0.0016666963134756303, 0.0012950393523015482, 0.0022288796333856469, 0.0013755408752780742, 0.0022065934850584029, 0.001720453052547579, 0.0010263138157518583, 0.0020451065304083146, 0.00092305921821682032, 0.0018187529538286496, 0.0015427919993230104, 0.0021568183075259201, 0.0016043302753607866, 0.00098964813039879125, 0.00088180450324806348, 0.0030356815878033491, 0.002779457792630224, 0.0022234673043779476, 0.0024782157843946948, 0.0029886543050390373, 0.0016838799658016588, 0.0029085569945471032, 0.0016813815631191934, 0.0017015898851302186, 0.0028932548763005205, 0.0024815622448178076, 0.0020584108935400953, 0.0026525698541270712, 0.0019350613628284817, 0.00077321298979012292, 0.0037685424955896693, 0.0030488496515402885, 0.0018076697545635954, 0.0035613590296346177, 0.0023530306078705771, 0.002706054506806502, 0.0020425667436432932, 0.0024920314686311747, 0.0032469783039800848, 0.0011147998034239798, 0.0010246854157871736, 0.0016960669960894446, 0.0024487814108826544, 0.0019147061648260738, 0.0028605983947445879, 0.0020116863274039793, 0.0025084364578659693]
pick_by_sent = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
if False:
    rouge_by_sent = [x for x, p in zip(rouge_by_sent, pick_by_sent) if p!=1]
    marginal_rouge_by_sent = [x for x, p in zip(marginal_rouge_by_sent, pick_by_sent) if p!=1]
    cos_by_sent = [x for x, p in zip(cos_by_sent, pick_by_sent) if p!=1]
    pick_by_sent = [x for x in pick_by_sent if x!=1]


redundancy_by_sent = [1 - mr/r for r, mr in zip(rouge_by_sent, marginal_rouge_by_sent)]
    
pick_by_sent_by_docs = [pick_by_sent[i:pick_by_sent.index(1, i + 1) if 1 in pick_by_sent[i + 1:] else len(pick_by_sent)] for i in range(len(pick_by_sent)) if pick_by_sent[i] == 1]
rouge_by_sent_iter = iter(rouge_by_sent)

rouge_by_sent_by_docs = [[next(rouge_by_sent_iter)for _ in doc] for doc in pick_by_sent_by_docs]
marginal_rouge_by_sent_iter = iter(marginal_rouge_by_sent)

marginal_rouge_by_sent_by_docs = [[next(marginal_rouge_by_sent_iter)for _ in doc] for doc in pick_by_sent_by_docs]
cos_by_sent_iter = iter(cos_by_sent)
cos_by_sent_by_docs = [[next(cos_by_sent_iter)for _ in doc] for doc in pick_by_sent_by_docs]

redundancy_by_sent_iter = iter(redundancy_by_sent)
redundancy_by_sent_by_docs = [[next(redundancy_by_sent_iter) for _ in doc] for doc in pick_by_sent_by_docs]


#Average
m = np.max(pick_by_sent)
cos_avg = [np.mean(l) for l in [[x for p, x in zip(pick_by_sent, cos_by_sent)  if p==i] for i in range(1, m)]]
rouge_avg = [np.mean(l) for l in [[x for p, x in zip(pick_by_sent, rouge_by_sent) if p==i ] for i in range(1, m)]]

plt.plot(cos_avg)
plt.plot(rouge_avg)
plt.figure()

print(cos_avg)
print(rouge_avg)

plt.figure()
plt.scatter(cos_by_sent, rouge_by_sent)
plt.xlabel("Cosine Similarity")
plt.ylabel("sentence ROUGE score")
plt.figure()
plt.scatter(cos_by_sent, marginal_rouge_by_sent)
plt.xlabel("Cosine Similarity")
plt.ylabel("marginal sentence ROUGE score")
plt.savefig('fig/greedy-marginal-rouge.png',bbox_inches='tight')

plt.figure()
for docx, docy in zip(pick_by_sent_by_docs, cos_by_sent_by_docs):
    plt.plot(docx, docy, '-',alpha=.3, lw=1)
plt.xlabel("Pick Numer")
plt.ylabel("Cosine Similarity")
#plt.xticks([1, 2, 3, 4])
#plt.xlim([.9, 4.1])
plt.savefig('fig/greedy-cos.png',bbox_inches='tight')
    
plt.figure()

for docx, docy in zip(pick_by_sent_by_docs, rouge_by_sent_by_docs):
    plt.plot(docx, docy, '-', alpha=.3, lw=1)
plt.xlabel("Pick Number")
plt.ylabel("ROUGE")
#plt.xlim([.9, 4.1])
#plt.xticks([1, 2, 3, 4])
plt.savefig('fig/greedy-rouge.png',bbox_inches='tight')
plt.figure()

for docx, docy in zip(pick_by_sent_by_docs, marginal_rouge_by_sent_by_docs):
    plt.plot(docx, docy, '-', alpha=.3, lw=1)
plt.xlabel("Pick Number")
plt.ylabel("Marginal ROUE")
#plt.xticks([1, 2, 3, 4])
#plt.xlim([.9, 4.1])
plt.figure()


for docx, docy in zip(pick_by_sent_by_docs, redundancy_by_sent_by_docs):
    plt.plot(docx, docy, '-', alpha=.3, lw=1)
plt.xlabel("Pick Numer")
plt.ylabel("Rouge Redundancy")
#plt.xticks([1, 2, 3, 4])
#plt.xlim([.9, 4.1])
plt.savefig('fig/greedy-redundancy.png',bbox_inches='tight')
    
plt.figure()

plt.scatter(cos_by_sent, marginal_rouge_by_sent)
from pandas import *
from pandas.plotting import scatter_matrix
df = DataFrame(np.asarray([rouge_by_sent, marginal_rouge_by_sent, cos_by_sent, pick_by_sent]).T, columns=['Rouge', 'Marginal Rouge', 'Cos', 'Pick Order'])

#scatter_matrix(df, alpha=0.2, figsize=(6, 6), diagonal='kde')

In [ ]:
str2vecs = [summarizer_modules.str2vec_avg, summarizer_modules.str2vec_arora, summarizer_modules.str2vec_doc2vec,summarizer_modules.str2vec_skipthought]

print(sentences[0][5])
rouge = [x[5]['rouge_1_recall'] for x in sentences]
#print(rouge)
plt.figure()
plt.hist(rouge)

corrs = [[sentences[j][1][i] for j in range(len(sentences))] for i in range(len(str2vec))]

corrs_with_avg = [[],[],[],[]]

print(len(sentences[0][5]))
for k, doc, doc_emb, sents, sent_embs in docs:
    doc_emb = [summarizer_modules.normalizeVec(np.mean(np.stack([x[i] for x in sent_embs], axis=1), axis=1)) for i in range(len(str2vecs))]
    for sent_emb in sent_embs:
        for i in range(len(doc_emb)):
            corrs_with_avg[i].append(np.inner(sent_emb[i], doc_emb[i]))

lens = [sent[2] for sent in sentences]
plt.figure()
plt.plot(lens, np.asarray(rouge), 'b.', alpha=.02)
plt.plot(np.unique(lens), np.poly1d(np.polyfit(lens, rouge, 1))(np.unique(lens)))
plt.xlim([0, 100]);plt.ylim(0, .4)

plt.figure()
plt.plot(lens, np.asarray(rouge)/np.asarray(lens), 'b.', alpha=.02)
plt.plot(np.unique(lens), np.poly1d(np.polyfit(lens, np.asarray(rouge)/np.asarray(lens), 1))(np.unique(lens)))
#plt.xlim([0, 100]);plt.ylim(0, .4)

from scipy.stats import pearsonr
import math

for i, corr in enumerate(corrs):
    corr = [x[0] for x in corr]
    log_rouge = [math.log(r + 1) for r in rouge ]
    plt.figure()
    plt.plot(corr, log_rouge, 'b.', alpha=.02)
    
    #Remove high-influence points
    if i >= 2:
        log_rouge = [r for r, c in zip(log_rouge, corr) if c > .4]
        corr = [c for c in corr if c > .4]
    
    plt.plot(np.unique(corr), np.poly1d(np.polyfit(corr, log_rouge, 1))(np.unique(corr)))
    plt.ylim([0, .4])
    plt.xlabel("Cosine Similarity ({})".format(names[str2vecs[i]]), fontsize=18)
    plt.ylabel("log(1 + ROUGE)", fontsize=18)
    plt.tight_layout()
    plt.savefig('fig/corr_{}.png'.format(names[str2vecs[i]]))
    print(pearsonr(corr, (log_rouge)))

In [ ]:
from summarizer_modules import *
import itertools
import random
import pandas as pd
from scipy.stats import ttest_rel

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

dist = []
results = pickle.load(open('results/results.pickle', 'rb'))

vectors = [str2vec_avg, str2vec_arora, str2vec_doc2vec, str2vec_skipthought]
selectors = [selector_greedy, selector_lexrank_continuous, selector_pca, selector_near_redundancy,
             selector_max_sim, selector_near_then_bf, selector_optimize_against_redundancy, selector_near, selector_cluster,
             selector_svm,
                     selector_random]
data = [['' for _ in range(len(vectors) + 1)] for _ in range(len(selectors))]
ci = [['' for _ in range(len(vectors) + 1)] for _ in range(len(selectors))]

docvec = [['' for _ in range(len(vectors) + 1)] for _ in range(len(selectors))]

r_random = results[selector_random, False, str2vec_doc2vec, '2004'][2]

for i, vector in enumerate(vectors):
    for j, selector in enumerate(selectors):
        try:
            r_false = results[(selector, False, vector, '2004')]
            r_true = results[(selector, True, vector, '2004')]
            r = r_false if r_false[0] > r_true[0] else r_true
            
            p = ttest_rel(r_random, r[2])[1]
            dist.extend(r[2])
            data[j][i + 1] = '{}{:.5f}({:.5f})(p={:.3f})'.format(bcolors.OKGREEN if p<=.05 else bcolors.FAIL, 
                                                               r[0] * 100, r[1] * 100, p)
            points = r[2]
            samples = sorted([np.mean([random.choice(points) for _ in range(len(points))]) for _ in range(10000)])
            ci[j][i + 1] = '{:.1f} - {:.1f}'.format(samples[24] * 100, samples[-25] * 100)
            docvec[j][i + 1] = '{} {}'.format(bcolors.OKGREEN if r_false[0] < r_true[0] else bcolors.FAIL, 100 *(r_true[0] - r_false[0]))
        except KeyError:
            pass
        data[j][0] = bcolors.HEADER + selector.__name__
        ci[j][0] = bcolors.HEADER + selector.__name__
        docvec[j][0] = bcolors.HEADER + selector.__name__
random_points = sum([results[(selector_random, True, vector, '2004')][2] for vector in vectors], [])
samples = sorted([np.mean([random.choice(random_points) for _ in range(len(random_points))]) for _ in range(10000)])
plt.hist(dist)
print('{:.1f} - {:.1f}'.format(samples[250] * 100, samples[-250] * 100))
from tabulate import tabulate
print(tabulate(data))
print(tabulate(ci))
print(tabulate(docvec))

In [ ]:
#Contour plot
import matplotlib
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

delta = 0.025
x = np.arange(0, 4.0, delta)
y = np.arange(0, 1.0, delta)
X, Y = np.meshgrid(x, y)
Z = [(x*x + 1 + 2*x*y) for x, y in zip(X, Y)]

levels = np.arange(np.min(Z), np.max(Z), 1.5)
plt.figure()
CS = plt.contour(X, Y, Z, colors='black', levels=levels)
plt.xlabel('i', fontsize=18)
plt.ylabel('Cosine Redundancy', fontsize=18)
plt.savefig('fig/contour.png')